In [16]:
from requests import Session
from bs4 import BeautifulSoup
import pandas as pd
import re

# Маскируемся под браузер
session = Session()
session.headers.update({ 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36' })
get = session.get

In [17]:
RUSFENCING_BASE_URL = 'http://rusfencing.ru'

def get_page(url):
    res = get(url)
    return BeautifulSoup(res.text, 'html.parser')

In [18]:
def get_fencers_page(num = 1):
    url = '{}/sportsmen.php?PAGEN_1={}'.format(RUSFENCING_BASE_URL, num)
    return get_page(url)

def get_fencers_pages_count():
    page = get_fencers_page()
    end = page.find('a', string='Конец')['href']
    r = re.search(r'PAGEN_1=(\d+)', end)
    if r and len(r.groups()) == 1:
        count = r.group(1)
        return int(count)
    else:
        return None

In [19]:
def get_rusfencing_id(cell):
    a = cell.find('a')['href']
    r = re.search(r'ID=(\d+)', a)
    if r and len(r.groups()) == 1:
        id = r.group(1)
        return int(id)
    else:
        return None

def normalize_degree(degree):
    # NOTE: Может, как-то в одну строку переписать?
    if not degree or degree == 'б/р':
        return None
    return degree

def get_all_fencers_on_page(num = 1):
    page = get_fencers_page(num)
    fencers = []
    rows = page.findAll('tr')
    for row in rows:
        cells = row.findAll('td')
        if len(cells) == 10:
            fencer = {
                'rusfencing_id': get_rusfencing_id(cells[1]),
                'license_number': cells[0].text,
                'name': cells[1].text,
                'sex': cells[2].text,
                'birthdate': cells[3].text,
                'weapon': cells[4].text,
                'degree': normalize_degree(cells[5].text),
                'subject': cells[6].text
            }
            fencers.append(fencer)
    return fencers

In [20]:
fencers = get_all_fencers_on_page(get_fencers_pages_count())
df = pd.DataFrame(fencers)
df.head(5)

TypeError: 'NoneType' object is not subscriptable